In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [0]:
df = spark.read.format('csv').option('header', True).load('/Volumes/workspace/default/test_practice/Data_Train.csv')

In [0]:
df.withColumn('Additional_Info', when((col('Additional_info').isNull()) & (col('Price')>5000) , lit('No Info')).otherwise(col('Additional_info'))).display()

In [0]:
df = df.withColumn(
    "Additional_Info",
    when((col("Additional_info").isNull()), lit("No Info")).otherwise(
        when(col("Total_Stops") == "non-stop", lit("No Info")).otherwise(
            col("Additional_info")
        )
    ),
)
df.display()

In [0]:
df.withColumn('info', expr("case when Additional_info is null then 'No Info' else Additional_info end")).display()

In [0]:
df = df.withColumn("splitted_col", split(col("Duration"), " ")).withColumn(
    "hour", col("splitted_col").getItem(0)
)
df.display()

In [0]:
df.withColumn('exploded_data', explode_outer(col('splitted_col'))).display()

In [0]:
help(explode)

In [0]:
help(explode_outer)

In [0]:
#from pyspark.sql.functions import array, arrays_zip, col

df = spark.createDataFrame([
    (1, [1, 2], ['a', 'b']),
    (2, [3, 4], ['c', 'd'])
], ['id', 'nums', 'letters'])

df_zipped = df.withColumn('zipped', arrays_zip(col('nums'), col('letters')))
display(df_zipped)

In [0]:
df_zipped.withColumn('map_val', map_from_entries(col('zipped'))).display()

In [0]:
df_zipped.withColumn('nums_new',col('zipped.nums')).withColumn('letters_new',col('zipped.letters')).display()

In [0]:
df_zipped.select(col('zipped.nums').alias('nums'), col('zipped.letters').alias('letters')).display()

In [0]:
df = df.filter((col('Total_Stops').isNotNull()) & (col('Additional_Info').isNotNull()))
df.withColumn('map_data', create_map(col('Total_Stops'), col('Additional_Info'))).display()

In [0]:
df_zipped.withColumn('array_sts', when(array_contains(col('letters'), 'b'), 'b').otherwise(lit('other'))).display()

In [0]:
df_zipped = df_zipped.select(map_from_entries(col('zipped')).alias('map_type'))

In [0]:
df_zipped = df_zipped.withColumn('exploded_col',explode(col('zipped')))

In [0]:
df_zipped.select('map_type',map_keys(col('map_type')).alias('map_key'), map_values(col('map_type')).alias('map_val')).display()

In [0]:
df = df.groupBy('Additional_Info','Total_Stops').agg(count('*').alias('count'), mean('price').alias('total_price'))
df.display()

In [0]:
#table----emp_name ,emp_id, gender
#you have to find count of total emp, count of male emp, count of female emp ---in a single query
#ouput format --- total_count male_count female_count

In [0]:
df.groupBy('Total_Stops').agg(collect_list(col('Additional_Info'))).display()

In [0]:
df.groupBy('Total_Stops').agg(collect_set(col('Additional_Info'))).display()

In [0]:
df.sort(col('count').desc()).display()

In [0]:
df.orderBy(col('Total_Stops').desc()).display()

## union

In [0]:
df_1 = spark.createDataFrame([(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd'), (5, 'e')], schema=['X','Y'])
df_2 = spark.createDataFrame([('a',2), ('b',7), ('c',9)], schema=['Y','X'])

In [0]:
df_1.union(df_2).display()

In [0]:
df_1.unionByName(df_2, allowMissingColumns=True).display()

In [0]:
df_1 = spark.createDataFrame([(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd'), (5, 'e')], schema=['X','Y'])
df_2 = spark.createDataFrame([('a',2,8), ('b',7,33), ('c',9,3)], schema=['Y','X','Z'])

In [0]:
df_1.union(df_2).display()

In [0]:
df_1.unionByName(df_2, allowMissingColumns=True).display()

## left semi and left anti

In [0]:
%sql
select * from table1 where col in (select col2 from table2)

In [0]:
df_1.join(df_2, on=[(df_1.Y==df_2.Y) & (df_1.X==df_2.X)], how='left_semi').display()

In [0]:
df_1.join(df_2, on=['Y'], how='left_semi').display()

In [0]:
df_1.join(df_2, on=[df_1.Y==df_2.Y], how='left_anti').display()

In [0]:
df.withColumn('Additional_Info', regexp_replace('Additional_Info',' ', '')).display()

In [0]:
'''task 1 --- #table----emp_name ,emp_id, gender
#you have to find count of total emp, count of male emp, count of female emp ---in a single query
#ouput format --- total_count male_count female_count
task 2--- join dept table with employee table
task 3 --- find the maximum salary for each department
'''

In [0]:
df.withColumn('hash_key', md5(concat_ws('_',col('Airline'),col('Source'),col('Destination')))).display()

In [0]:
df1.join(df2, on=['dept_id','emp_id','dept_name'], how='inner').display()


In [0]:
df.display()

In [0]:
df.write.format('delta').mode('overwrite').save('/Volumes/workspace/default/test_practice/target_data')

In [0]:
df.write.format('delta').mode('overwrite').option('mergeSchema', True).save('/Volumes/workspace/default/test_practice/target_data')

In [0]:
df.write.format('parquet').save('/Volumes/workspace/default/test_practice/target_record.parquet')